<h2>Note: Most of the code in this notebook will either run and hang indefinitely, error out, or simply has not been tested!</h2>

This notebook was used for testing out a few different libraries, and as a staging area for some of the residualization & interaction term generation methodology we were going to try to utilize. Most of the DAG-generating code will not run, and the functions near the bottom were never tested. This is maintained for record. I may also tap into the redisualization / interaction functions at work to see how feasible it might be to productize a more causally-driven Marketing-Mix Model (MMM)!

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import pickle
from sklearn.preprocessing import KBinsDiscretizer
from causallearn.search.ConstraintBased.PC import pc
from causallearn.search.ScoreBased.GES import ges
from causallearn.utils.PDAG2DAG import pdag2dag
from dowhy import gcm
from sklearn.linear_model import LinearRegression, LogisticRegression, PoissonRegressor
import statsmodels.api as sm
from itertools import combinations
import networkx as nx
import warnings
from IPython.display import display

BASE_DIR = Path().resolve()

warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    module="causallearn"
)

warnings.filterwarnings("ignore", message=".*", category=FutureWarning, module="pandas")

get_ipython().kernel.shell.displayhook.flush()

In [ ]:
df = pd.read_csv(BASE_DIR / "natality_7yr_test_data_for_dag.csv")
df['date'] = pd.to_datetime(df['date'])

df

In [ ]:
sampled = (
    df.groupby(df['date'].dt.year, group_keys=False)
      .apply(lambda g: g.sample(n=5000, random_state=42))
      .reset_index(drop=True)
)

In [ ]:
continuous_cols_to_transform = [
    'bmi', 'time_sin', 'time_cos'
]

sampled[continuous_cols_to_transform].describe()

Since most of our variables are one-hot / binary, we're going to convert them to bins so we can use one algorithm to generate & explore the DAG.

In [ ]:
# We're dropping these because we have discrete versions of them, which might be more informative for the DAG.
binary_cols_to_drop = ["cig_0_binary", "cig_1_binary", "cig_2_binary", "cig_3_binary", "priorlive_binary", "priordead_binary", "priorterm_binary", "precare_binary"]
df_discrete = sampled.drop(columns=binary_cols_to_drop)

discretizer = KBinsDiscretizer(
    n_bins=4,
    encode='ordinal',
    strategy='quantile',
    quantile_method='averaged_inverted_cdf'
)

df_discrete[continuous_cols_to_transform] = discretizer.fit_transform(df_discrete[continuous_cols_to_transform])
df_discrete[continuous_cols_to_transform] = df_discrete[continuous_cols_to_transform].astype('int32')

X = df_discrete.drop(columns=["date"]).astype(int).to_numpy()
names = df_discrete.drop(columns=["date"]).columns.tolist()

In [ ]:
describe_df = df_discrete.describe()

describe_df.to_csv("describe_df.csv")

In [ ]:
cg1 = ges(
    X,
    score_func="local_score_BDeu", # or "bic"
    node_names=names
)

with open("ges.pkl", "wb") as f:
    pickle.dump(cg1, f)

In [ ]:
cg2 = pc(
    X,
    indep_test="gsq",
    alpha=0.05,
    stable=False,
    node_names=names
)

with open("pc.pkl", "wb") as f:
    pickle.dump(cg2, f)

For either above, must convert from Partially Direct Acyclic Graph to Direct Acyclic Graph for use with DoWhy

In [ ]:
def causallearn_to_networkx(adjacency_matrix, node_names):
    """
    Convert causal-learn adjacency matrix to NetworkX DiGraph
    """
    G = nx.DiGraph()
    
    # Add nodes
    for name in node_names:
        G.add_node(name)
    
    # Add edges where adjacency_matrix[i,j] != 0 means i -> j
    for i in range(len(adjacency_matrix)):
        for j in range(len(adjacency_matrix)):
            if adjacency_matrix[i, j] == -1 and adjacency_matrix[j, i] == 1:
                # i -> j (tail at i, head at j)
                G.add_edge(node_names[i], node_names[j])
    
    return G

In [ ]:
# Convert PDAG to DAG
dag = pdag2dag(cg.G)

adjacency_matrix = dag.graph

nx_graph = causallearn_to_networkx(adjacency_matrix, names)

pgmpy HillClimbSearch + BDeu

In [ ]:
from pgmpy.estimators import HillClimbSearch, BDeuScore

hc = HillClimbSearch(df, scoring_method=BDeuScore(df, equivalent_sample_size=1))
model = hc.estimate()  # returns a DAG

pgmpy Max-Min Hill-Climb

In [ ]:
from pgmpy.estimators import maxmin

<h2>Explore</h2>

<h2>Clean Causal Relationships and Build Interaction Terms</h2>

- Use PCM when you only need interventions/predictions and want maximum flexibility in mechanism choice
- Use SCM when you need counterfactual reasoning ("what if X had been different?")

Since we're using AdditiveNoiseModel, PCM is more or less the same as SCM.

AdditiveNoiseModel

AdditiveNoiseModel represents a continuous functional causal model of the form Y = f(X) + N, where X is the input (typically the direct causal parents of Y) and the noise N is assumed to be independent of X.

Each variable Y is modeled as: Y = (linear combination of parents) + independent noise.

When you call gcm.fit(pcm, df), it:

1. Fits a linear regression from parents → child for each node
2. Computes residuals to learn the noise distribution
3. Stores both the regression model and noise model



3 main stages:

DAG-Informed

- Partial out: Partialing-out (residualization) removes confounding and mediation bias, but it only projects away linear additive relationships between each variable and its parents. It can’t invent or preserve interaction structure. Residualization step protects causal identification but not functional completeness.

- Add Interactions: captures moderation or how one cause modifies another’s impact on Y. Must be direct parents of Y, not ancestors or descendents. If the don't directly cause Y, they’re outside Y’s equation.

Optimization

- Feature Selection: MOO for AIC, Precision and Condition Number. Analyze Pareto Front, and choose feature set based on model fit.

In [ ]:
def _is_binary(s):
    vals = pd.Series(s).dropna().unique()
    return len(vals) <= 2 and set(vals).issubset({0,1,True,False})

def _is_count(s):
    s_nonan = pd.Series(s).dropna()
    return (s_nonan.dtype.kind in "iu") and (s_nonan.min() >= 0)

def fit_linear_gcm(G_nx, df):
    pcm = gcm.ProbabilisticCausalModel(G_nx)
    for v in G_nx.nodes():
        y = df[v]
        if _is_binary(y):
            mech = LogisticRegression(max_iter=1000)
        elif _is_count(y):
            mech = PoissonRegressor(alpha=0.0, max_iter=1000)
        else:
            mech = LinearRegression()
        pcm.set_causal_mechanism(v, gcm.AdditiveNoiseModel(mech))
    gcm.fit(pcm, df)
    return pcm

def _predict_any(model, X):
    # logistic uses expected value (prob of class 1)
    if hasattr(model, "predict_proba"):
        return model.predict_proba(X)[:, 1]
    # linear, etc.
    return model.predict(X)

def residualize_all_nodes(pcm, G_nx, df, n_splits=5, random_state=42):
    df_out = df.copy()
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    for v in G_nx.nodes():
        ps = list(G_nx.predecessors(v))
        if not ps:
            continue

        X = df[ps].values
        y = df[v].values
        preds = np.zeros(len(df))

        # clone the already-chosen mechanism type from pcm for consistency
        base_model = pcm.causal_mechanism(v).prediction_model

        for train_idx, test_idx in kf.split(X):
            # fresh clone to avoid leakage across folds
            model = type(base_model)(**getattr(base_model, "get_params", lambda: {})())
            model.fit(X[train_idx], y[train_idx])
            preds[test_idx] = _predict_any(model, X[test_idx])

        df_out[f"{v}_resid"] = y - preds

    return df_out

def parents(G, v):
    return list(G.predecessors(v))

def _resid_or_raw(df, col):
    return f"{col}_resid" if f"{col}_resid" in df.columns else col

def candidate_interactions_for_Y(G, Y):
    """
    GeneratesDAG-valid interactions, aka unordered pairs among parents of
    y that are not ancestor–descendant.
    """
    ps = parents(G, Y)
    pairs = []
    for a, b in combinations(ps, 2):
        if nx.has_path(G, a, b) or nx.has_path(G, b, a):
             # skips hierarchical pairs
            continue
        pairs.append((a, b))
    return pairs

def add_interactions(df, pairs):
    """
    Mean-center inputs, then create products for each (a,b).
    """
    df2 = df.copy()
    # collect the columns we’ll use (residualized if they exist)
    cols = sorted({ _resid_or_raw(df2, a) for ab in pairs for a in ab })
    # mean-center to stabilize and make main effects interpretable at the mean
    centered = df2[cols] - df2[cols].mean(axis=0)
    for a, b in pairs:
        ca, cb = _resid_or_raw(df2, a), _resid_or_raw(df2, b)
        df2[f"{a}__x__{b}"] = centered[ca] * centered[cb]
    return df2

def design_for_Y(G, df_resid, Y):
    """
    Creates the cleaned, augmented design matrix and returns X (with intercept) and y.
    """
    ps = parents(G, Y)
    base_cols = [_resid_or_raw(df_resid, p) for p in ps]

    # main effects (centered)
    X_base = df_resid[base_cols].copy()
    X_base = X_base - X_base.mean(axis=0)

    # interactions among valid co-parents
    pairs = candidate_interactions_for_Y(G, Y)
    X_all = add_interactions(pd.concat([X_base], axis=1), pairs)

    # assemble final design
    covs = list(X_base.columns) + [f"{a}__x__{b}" for (a, b) in pairs]
    X = sm.add_constant(X_all[covs], has_constant="add")
    y = df_resid[Y]
    return X, y

In [ ]:
pcm = fit_linear_gcm(G_nx, df)
df_resid = residualize_all_nodes(pcm, G_nx, df)

X_tot, y = design_for_Y(G_nx, df_resid, 'no_mmorb')
res_tot = sm.OLS(y, X_tot).fit()